#### Código - Función Extraer Consulta de Datos sobre Operación Financiera CREDIFIEL

El presente código tiene como objetivo crear una visualización final de la consulta de datos sobre operaciones financieras 

In [2]:
# Importar librerías
import json
import matplotlib.pyplot as plt
from matplotlib import rcParams
import psycopg2
import os
import re
import csv
import platform
import subprocess


In [3]:
# Parámetros de conexión
DB = {
    'host': '192.168.252.38',
    'database': 'biprod',
    'user': 'alejandra_gonzalez',
    'password': 'AGonzalez123.$'
}

try:
    # Conexión a la base de datos
    conn = psycopg2.connect(**DB)
    cur = conn.cursor()

    # Verificación de la conexión
    print("Conexión exitosa a la base de datos 🥳✅")

    # Consulta SQL
    query = """
    SELECT 
        l.usuarioCapturo,
        l.nombreEmpleado,
        l.puestoEmpleado,
        to_char(l.fechaSolicitud, 'DD/MM/YYYY') AS fechaSolicitud,
        COALESCE(NULLIF(l.region, 'NACIONAL'), 'CIUDAD DE MEXICO') AS region,
        COALESCE(NULLIF(l.estado, 'CALL CENTER'), 'CORPORATIVO') AS estado,
        COALESCE(NULLIF(l.sucursal, 'BROKER MEXI'), 'CORPORATIVO') AS sucursal,
        COALESCE(
            (SELECT UPPER(r.descripcion)
             FROM acle.erp_catsucursales s
             JOIN acle.erp_catregion r ON s.idregion = r.idregion
             WHERE UPPER(s.nombresucursal) = UPPER(
                COALESCE(p.sucursal, pp.sucursal, ppp.sucursal, 'CORPORATIVO'))
             AND fechainactividad IS NULL 
             LIMIT 1), 
            'CORPORATIVO'
        ) AS regionAsesor,
        COALESCE(p.estado, pp.estado, ppp.estado, 'CIUDAD DE MEXICO') AS estadoAsesor,
        COALESCE(p.sucursal, pp.sucursal, ppp.sucursal, 'CORPORATIVO') AS sucursalAsesor,
        l.numeroSolicitud,
        l.numeroCliente,
        l.nombreCliente,
        l.folioSolicitud,
        dh.description AS dependenciaHomologada,
        cp.lineaNegocio,
        cp.descripcionCOE,
        l.tipoDispersion,
        l.tipoPromocion,
        l.pagos,
        split_part(l.nombreGerente, ',', 1) AS cvegerente,
        TRIM(split_part(l.nombreGerente, ',', 2)) AS nombreGerente,
        split_part(l.nombreSupervisor, ',', 1) AS cvesupervisor,
        TRIM(split_part(l.nombreSupervisor, ',', 2)) AS nombreSupervisor,
        split_part(l.nombreAsesor, ',', 1) AS cveasesor,
        TRIM(split_part(l.nombreAsesor, ',', 2)) AS nombreAsesor,
        to_char(b.fechaApertura, 'DD/MM/YYYY') AS fechaApertura,
        (l.montoCobrado + l.montoACobrar) AS montoColocacion,
        l.fechaCobro,
        l.numeroCredito,
        l.estatusC,
        l.estatusD,
        l.estatus,
        l.fechaHoraInicioCaptura,
        l.fechaHoraFinCaptura,
        l.fechaEstatus,
        l.horaEstatus,
        l.horasOperativas,
        l.totalDevolucionesZell,
        l.totalDevolucionesAnalistas,
        l.lineaCredito,
        l.seguro,
        l.banco,
        l.subtipoPromocion,
        l.producto,
        b.dap AS cveDap,
        (SELECT od.etiqueta
         FROM dwhprod.dwh_bidatocomun dc
         JOIN dwhprod.dwh_catopciondato od ON dc.iddato = od.iddato AND dc.valordato = od.valor
         WHERE dc.numerocredito = l.numeroSolicitud
           AND dc.idreferencia = 9
           AND dc.iddato = 95
         LIMIT 1) AS tipoNombramiento,
        (SELECT od.etiqueta
         FROM dwhprod.dwh_bidatocomun dc
         JOIN dwhprod.dwh_catopciondato od ON dc.iddato = od.iddato AND dc.valordato = od.valor
         WHERE dc.numerocredito = l.numeroSolicitud
           AND dc.idreferencia = 9
           AND dc.iddato = 96
         LIMIT 1) AS tipoEmpleado,
        l.genero,
        p.puestoAsesor,
        CASE 
            WHEN t.tasaGlobal IS NOT NULL 
            THEN CONCAT_WS('', TO_CHAR(CAST(t.tasaGlobal AS NUMERIC), '99D0999'), '%') 
            ELSE '' 
        END AS valorTasaGlobal,
        l.dependencia
    FROM 
        dwhprod.dwh_vwTiemposOriginacion2 l
    LEFT JOIN dwhprod.dwh_bicredito b ON l.numeroSolicitud = b.numeroSolicitud
    LEFT JOIN dwhprod.dwh_biplantillaempleados p ON split_part(l.nombreAsesor, ',', 1) = p.idEmpleado
    LEFT JOIN dwhprod.dwh_biplantillaempleados pp ON split_part(l.nombreSupervisor, ',', 1) = pp.idEmpleado
    LEFT JOIN dwhprod.dwh_biplantillaempleados ppp ON split_part(l.nombreGerente, ',', 1) = ppp.idEmpleado
    LEFT JOIN dwhprod.dwh_catDependencias dd ON b.idDependencia = dd.idDependencia
    LEFT JOIN dataprod.catafiliatehm dhm ON UPPER(dd.cveDependencia) = UPPER(dhm.afiliateid)
    LEFT JOIN dataprod.catgovermentdependency dh ON dhm.govermentdependencyid = dh.govermentdependencyid
    LEFT JOIN dwhprod.dwh_catproductos cp ON b.idProducto = cp.idProducto
    LEFT JOIN dwhprod.tblcreddictaminacion t ON b.numeroCredito = t.idCredito;
    """
    
    # Ejecutar la consulta
    cur.execute(query)

    # Recuperar los resultados
    rows = cur.fetchall()
    column_names = [desc[0] for desc in cur.description]

    # Guardar resultados en un archivo CSV
    with open('consulta_resultados.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(column_names)  # Escribir encabezados
        writer.writerows(rows)         # Escribir datos

    print("Consulta ejecutada y resultados guardados en 'consulta_resultados.csv'")

except psycopg2.Error as e:
    print(f"Error en la conexión o consulta: {e}")

finally:
    if cur:
        cur.close()
    if conn:
        conn.close()

Conexión exitosa a la base de datos 🥳✅
Consulta ejecutada y resultados guardados en 'consulta_resultados.csv'


In [3]:
output_file = 'consulta_resultados.csv'
absolute_path = os.path.abspath(output_file)
print(f"El archivo se guardó en la ruta: {absolute_path}")

El archivo se guardó en la ruta: c:\Users\quetz\AppData\Local\Programs\Microsoft VS Code\consulta_resultados.csv


In [53]:
# Ruta relativa del archivo CSV (debe estar en el mismo directorio que tu script o notebook)
csv_path = 'consulta_resultados.csv'

In [56]:
# Leer el archivo en un DataFrame de pandas
df = pd.read_csv(csv_path, encoding='utf-8', on_bad_lines='skip')  # Usa 'latin1' si hay problemas con la codificación
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 52 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   usuariocapturo              22 non-null     object 
 1   nombreempleado              2 non-null      object 
 2   puestoempleado              2 non-null      object 
 3   fechasolicitud              22 non-null     object 
 4   region                      22 non-null     object 
 5   estado                      22 non-null     object 
 6   sucursal                    22 non-null     object 
 7   regionasesor                22 non-null     object 
 8   estadoasesor                22 non-null     object 
 9   sucursalasesor              22 non-null     object 
 10  numerosolicitud             22 non-null     int64  
 11  numerocliente               22 non-null     int64  
 12  nombrecliente               22 non-null     object 
 13  foliosolicitud              22 non-nu

In [57]:
df.head()

,usuariocapturo,nombreempleado,puestoempleado,fechasolicitud,region,estado,sucursal,regionasesor,estadoasesor,sucursalasesor,...,banco,subtipopromocion,producto,cvedap,tiponombramiento,tipoempleado,genero,puestoasesor,valortasaglobal,dependencia
0,us_odApp,NaN,NaN,02/01/2025,SURESTE,OAXACA,OAXACA,SURESTE,OAXACA,OAXACA,...,SANTANDER,TRADICIONAL,CFNT29120Q,NaN,BASE,Activo,Masculino,ASESOR DE CREDITO,NaN,SNTE 22 OAXACA
1,us_odApp,NaN,NaN,02/01/2025,SURESTE,OAXACA,HUAJUAPAN,SURESTE,OAXACA,HUAJUAPAN,...,SANTANDER,REMODELA,CFRM25_36Q,NaN,BASE,Activo,Femenino,ALIADO COMERCIAL,NaN,SNTE 22 OAXACA
2,us_odApp,NaN,NaN,02/01/2025,GOLFO,QUINTANA ROO,CANCUN,GOLFO,QUINTANA ROO,CANCUN,...,BBVA BANCOMER,TRADICIONAL,CFLQ52120Q,NaN,BASE,Activo,Masculino,ALIADO COMERCIAL,NaN,SNTE 25 QUINTANA ROO
3,us_odApp,NaN,NaN,02/01/2025,CORPORATIVO,NACIONAL,CALL CENTER,GOLFO,VERACRUZ,XALAPA,...,HSBC,TRADICIONAL,CFNT98_96Q,NaN,BASE,Activo,Masculino,ASESOR DE CREDITO,NaN,VER SAL SNTSA SEG POP (CR095)
4,us_odApp,NaN,NaN,02/01/2025,CORPORATIVO,NACIONAL,CALL CENTER,CORPORATIVO,CIUDAD DE MEXICO,CORPORATIVO,...,SANTANDER,TRADICIONAL,CF77OD_96Q,NaN,BASE,Activo,Femenino,ASESOR DE CREDITO DIGITAL,NaN,SNTE 22 OAXACA


In [ ]:
#Crear un groupby por el tipo de estatus del financiamiento